<a href="https://colab.research.google.com/github/allen60725/GLM_Pricing/blob/master/Auto_PurePremium_Tweedie_Rev01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pure Premium Modelling with Tweedie GLM

## Loading data
Let us load the datasets and take a look at them

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [0]:
# The data to load
f = "/content/gdrive/My Drive/NextGenModeling/Bi_modeling_binned_base.csv"

# As this is a demo, the large data set was reduced by taking every N-th (in this case 5th) 
n = 5

# Count the lines or use an upper bound
num_lines = sum(1 for l in open(f))

# The row indices to skip - make sure 0 is not included to keep the header!
skip_idx = [x for x in range(1, num_lines) if x % n != 0]

# Read the data
df= pd.read_csv(f, skiprows=skip_idx)

In [4]:
df.shape


(103742, 126)

In [5]:
df.head()

,polyr,ModelYr,bi_ecy,bi_wp,EstimatedAnnualDistance_v2,yrs_driving_exp,bi_SFX_CNT,SFX_CNT,bi_nc_inc_loss_dcce_t_100K,bi_nc_inc_loss_dcce_t,...,veh_op_matrix_9_1,veh_op_matrix_9_2,veh_op_matrix_9_3,veh_op_matrix_9_4,veh_op_matrix_9_5,veh_op_matrix_9_6,veh_op_matrix_missing,DrvGender_v2 _Female,DrvGender_v2 _Male,DrvGender_v2 _Missing
0,2011,2003,0.5,137.450394,6000,53.845514,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,2013,2003,0.5,165.416589,6000,53.845514,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
2,2016,2003,0.5,165.416589,5000,58.878818,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
3,2009,2006,0.5,137.450394,5000,53.845514,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,2011,2006,0.5,137.450394,3001,53.845514,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0


**Data Preparation**



*   Create target variables
*   Clean up  variables not for modeling



In [0]:
df['frequency']=df['bi_SFX_CNT']/df['bi_ecy']

df['severity']=df['bi_nc_inc_loss_dcce_t_100K']/df['bi_SFX_CNT']

df['ppremium']=df['bi_nc_inc_loss_dcce_t_100K']/df['bi_ecy']

In [7]:
list(df)

['polyr',
 'ModelYr',
 'bi_ecy',
 'bi_wp',
 'EstimatedAnnualDistance_v2',
 'yrs_driving_exp',
 'bi_SFX_CNT',
 'SFX_CNT',
 'bi_nc_inc_loss_dcce_t_100K',
 'bi_nc_inc_loss_dcce_t',
 'risk_zip_5',
 'DrvPointsCharged_V2',
 'Last_Owner_Average_Miles',
 'Last_Owner_Recent_Annual_Mileage',
 'Last_Owner_California_Recent_Ann',
 'Modeled_Annual_Mileage',
 'annual_mileage_v1',
 'annual_mileage_v2',
 'id',
 'VehAge',
 'ClaimInd',
 'PolicyProgramCode_Affinity Group',
 'PolicyProgramCode_Civil Servant',
 'PolicyProgramCode_Educator',
 'PolicyProgramCode_Firefighter',
 'PolicyProgramCode_Law Enforcement',
 'PolicyProgramCode_Non-Civil Servant',
 'VehType_Antique',
 'VehType_MotorHome',
 'VehType_PrivatePassengerAuto',
 'NewOrUsedInd_Missing',
 'NewOrUsedInd_New',
 'NewOrUsedInd_Used',
 'gooddriverind_v2_Missing',
 'gooddriverind_v2_N',
 'gooddriverind_v2_Y',
 'DrvMaritalStatus_v2_Married',
 'DrvMaritalStatus_v2_Missing',
 'DrvMaritalStatus_v2_Single',
 'DrvGender_v2_Female',
 'DrvGender_v2_Male',
 'b

In [0]:
cols = [c for c in df.columns if c.lower()[:13] != 'veh_op_matrix']

df = df[cols]

cols = [c for c in df.columns if c.lower()[:14] != 'bi_split_limit']

df = df[cols]

cols = [c for c in df.columns if c.lower()[:21] != 'bi_nc_inc_loss_dcce_t']

df = df[cols]



**Modeling:**


*   Assign independent variables  

*   Select target variable and call model function with matching distribution family 



In [0]:
X=df.drop(['bi_wp','ppremium','bi_SFX_CNT','SFX_CNT','ClaimInd', 'frequency','severity'], axis = 1)

In [0]:
# three target variables: ppremium, serverity, frequency are modeled 
# with three different distributions: Tweedie, Possion, Gamma, respectively

def AutoGLM(Target, X):
  y=df[Target]
  if Target =='frequency':   
    family = sm.families.Poisson(var_power=1.5)
 
  elif Target =='severity':
    family = sm.families.Gamma(var_power=1.5)
 
  else:
    family = sm.families.Tweedie(var_power=1.5)
    
  model = sm.GLM(exog = X, endog = y, family=family)
  report = model.fit()
  return report, report.summary()

In [11]:
# As an example here, when pure premium ('ppremium') is seleted, GLM function with Tweedie GLM was called.

AutoGLM("ppremium", X)

(<statsmodels.genmod.generalized_linear_model.GLMResultsWrapper at 0x7f4614b5bf98>,
 <class 'statsmodels.iolib.summary.Summary'>
 """
                  Generalized Linear Model Regression Results                  
 Dep. Variable:               ppremium   No. Observations:               103742
 Model:                            GLM   Df Residuals:                   103711
 Model Family:                 Tweedie   Df Model:                           30
 Link Function:                    log   Scale:                          10157.
 Method:                          IRLS   Log-Likelihood:                    nan
 Date:                Thu, 02 May 2019   Deviance:                   8.0458e+06
 Time:                        06:25:54   Pearson chi2:                 1.05e+09
 No. Iterations:                   100   Covariance Type:             nonrobust
                                           coef    std err          z      P>|z|      [0.025      0.975]
 ----------------------------------------